# Task2: Sentiment Analysis 

In [1]:
#!pip install -r requirements.txt

## Introduction

RoBERTa:

> Why: RoBERTa has shown excellent performance across various NLP tasks, often outperforming BERT due to its robust optimization techniques and larger training corpus. It handles nuances in text very well, which is crucial for sentiment analysis.
    Strengths: Strong performance on a wide range of tasks, robust pre-training.

DistilBERT:

> Why: DistilBERT is a distilled version of BERT that retains much of BERT's performance while being faster and more resource-efficient. It provides a good trade-off between performance and computational efficiency.
    Strengths: Faster and lighter than BERT, making it suitable for applications where speed and resource usage are considerations.

Electra:

> Why: Electra uses a different pre-training approach where the model learns to distinguish real input tokens from fake ones generated by another model. This approach is computationally efficient and leads to strong performance.
    Strengths: Computationally efficient, strong performance, particularly effective on downstream tasks after pre-training.

Train and Evaluate All Models:

> Train each model with default hyperparameters and evaluate their performance on the validation set.

Select the Best Model:

> Select the model with the highest F1 score on the validation set.

Fine-Tune the Best Model Using Optuna:

> Use Optuna to optimize the hyperparameters for the best-performing model.

Store All Models and Tokenizers:

> Store the trained models and their tokenizers in the models dictionary for use during prediction.

Predict Function:

> The predict function takes a model, tokenizer, and text input, and returns the predicted label.

Predict Sample Function:

> The predict_sample function takes a sample index, retrieves the sample text, and gets predictions from all models, including the fine-tuned model.
    The function returns the text, ground truth label, and predictions from each model.

Gradio Interface:

> Use the Gradio interface to select a sample and display predictions from all models, including the fine-tuned model.

## Imports

In [2]:
import os
import re
import json
import yaml
import nltk
import torch
import optuna
import logging
import numpy as np
import gradio as gr
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
from transformers import (
    AutoTokenizer,
    RobertaForSequenceClassification,
    DistilBertForSequenceClassification,
    ElectraForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Sentiment-Analysis')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data

In [3]:
# Load the data
train_df = pd.read_csv('./data/Sentiment_Training.csv', sep=';')
test_df = pd.read_csv('./data/Sentiment_Test.csv', sep=';')

In [4]:
# Load abbreviations from the YAML file
with open('./data/abbreviations.yaml', 'r') as file:
    abbreviations = yaml.safe_load(file)

def replace_abbreviations(text):
    for abbr, full_form in abbreviations.items():
        text = text.replace(abbr, full_form)
    return text

# Replace abbreviations in test data
train_df['text'] = train_df['text'].apply(replace_abbreviations)
test_df['text'] = test_df['text'].apply(replace_abbreviations)

## EDA

In [5]:
# Display the first few rows and information of the training dataset
logger.info(f"\nFirst few rows of the training dataset:\n{train_df.head()}")

INFO:Sentiment-Analysis:
First few rows of the training dataset:
   label                                               text
0      4  I stalk this tareyouck.  I've been to indyoust...
1      2  who areeally knows if this is good pho oare no...
2      4  I LOVE Bloom Salon... all of theiare stylinked...
3      0  We wearee epreviously named twittercited to ea...
4      2  So this is a place, with food. That myouch can...


In [6]:
logger.info("\nInformation about the training dataset:")
train_df.info()

INFO:Sentiment-Analysis:
Information about the training dataset:


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   10000 non-null  int64 
 1   text    10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [7]:
# Calculate text length
train_df['text_length'] = train_df['text'].apply(len)
logger.info("\nStatistics of text lengths in the training dataset:")
logger.info(f"\n{train_df['text_length'].describe()}")

# Histogram of text lengths using Plotly
fig = px.histogram(train_df, x='text_length', nbins=50, title='Distribution of Text Lengths')
fig.update_layout(
    xaxis_title='Text Length',
    yaxis_title='Frequency',
    bargap=0.2
)
fig.show()

INFO:Sentiment-Analysis:
Statistics of text lengths in the training dataset:
INFO:Sentiment-Analysis:
count    10000.000000
mean       968.686800
std        868.751153
min          2.000000
25%        376.000000
50%        723.000000
75%       1279.000000
max       6916.000000
Name: text_length, dtype: float64


In [8]:
# Distribution of labels
logger.info("\nDistribution of labels in the training dataset:")
logger.info(f"\n{train_df['label'].value_counts().sort_index()}")

# Plot of label distribution using Plotly
label_distribution = train_df['label'].value_counts().sort_index()
fig = px.bar(label_distribution, title='Distribution of Labels')
fig.update_layout(
    xaxis_title='Label',
    yaxis_title='Count',
    xaxis=dict(tickmode='linear')
)
fig.show()

INFO:Sentiment-Analysis:
Distribution of labels in the training dataset:
INFO:Sentiment-Analysis:
label
0    2035
1    1977
2    1943
3    1991
4    2054
Name: count, dtype: int64


In [9]:
# Function to clean and tokenize text
def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Tokenize all words
stop_words = set(["a", "an", "the", "and", "or", "but", "if", "in", "on", "with", "as", "of", "at", "by", "for", "from", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])
all_words = train_df['text'].apply(tokenize).sum()

# Find most common words
word_counts = Counter(all_words)
common_words = word_counts.most_common(20)

# Generate WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counts)

# Plot WordCloud using Matplotlib
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Common Words in Texts')
plt.show()

In [10]:
# Text lengths grouped by label and described
text_length_by_label = train_df.groupby('label')['text_length'].describe()

# Boxplot of text lengths by label using Plotly
fig = px.box(train_df, x='label', y='text_length', title='Text Lengths by Label')
fig.update_layout(
    xaxis_title='Label',
    yaxis_title='Text Length'
)
fig.show()

# Output text lengths by label
logger.info(f"\nText lengths by label:\n{text_length_by_label}")

INFO:Sentiment-Analysis:
Text lengths by label:
        count         mean         std   min    25%    50%     75%     max
label                                                                     
0      2035.0  1075.465356  988.724380   2.0  398.0  766.0  1421.5  6656.0
1      1977.0  1076.401619  937.309625  31.0  433.0  809.0  1431.0  6836.0
2      1943.0  1013.536284  855.510465   7.0  429.0  792.0  1341.0  6916.0
3      1991.0   900.193872  747.528743  14.0  361.5  709.0  1221.0  6562.0
4      2054.0   783.185492  751.262168   8.0  292.0  549.5  1003.0  6825.0


## Implementation

In [11]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    logger.info(f"Using device: {torch.cuda.get_device_name(0)}")

model_names = {
    'DistilBERT': ('distilbert-base-uncased', DistilBertForSequenceClassification),
    'Electra': ('google/electra-base-discriminator', ElectraForSequenceClassification),
    'RoBERTa': ('roberta-base', RobertaForSequenceClassification)
}

INFO:Sentiment-Analysis:Using device: NVIDIA GeForce RTX 4080


In [12]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [13]:
def validate_labels(labels):
    unique_labels = set(labels)
    logger.info(f"Unique labels: {unique_labels}")
    assert all(label in [0, 1, 2, 3, 4] for label in unique_labels), "Labels are out of the expected range."

def compute_metrics(p):
    predictions = np.array(p.predictions)
    preds = np.argmax(predictions, axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'eval_accuracy': accuracy,
        'eval_precision': precision,
        'eval_recall': recall,
        'eval_f1': f1
    }

def plot_confusion_matrix(labels, preds, model_name):
    conf_mat = confusion_matrix(labels, preds)
    z = conf_mat
    x = ['0', '1', '2', '3', '4']
    y = ['0', '1', '2', '3', '4']
    
    z_text = [[str(y) for y in x] for x in z]

    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Blues')
    fig.update_layout(
        title_text=f'Confusion Matrix for {model_name}',
        xaxis=dict(title='Predicted Labels'),
        yaxis=dict(title='True Labels')
    )
    fig.show()

In [14]:
def train_model(model_name, train_df, learning_rate=5e-5, batch_size=8, num_epochs=3, use_early_stopping=False, weight_decay=0.01, gradient_accumulation_steps=4):
    logger.info(f"Training model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_names[model_name][0])
    model_class = model_names[model_name][1]
    model = model_class.from_pretrained(model_names[model_name][0], num_labels=5)

    # Add an identifier to the model config
    model.config.model_id = f"{model_name}_trained"

    # Preprocess data and split into training and validation sets
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_df['text'].tolist(), train_df['label'].tolist(), test_size=0.2, random_state=42, stratify=train_df['label'].tolist())

    validate_labels(train_labels)
    validate_labels(val_labels)

    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

    train_dataset = SentimentDataset(train_encodings, train_labels)
    val_dataset = SentimentDataset(val_encodings, val_labels)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="epoch",  # Set save strategy to match evaluation strategy
        learning_rate=learning_rate,
        load_best_model_at_end=use_early_stopping,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        gradient_accumulation_steps=gradient_accumulation_steps
    )

    data_collator = DataCollatorWithPadding(tokenizer)

    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)] if use_early_stopping else []

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=callbacks
    )

    trainer.train()
    eval_results = trainer.evaluate()
    predictions = trainer.predict(val_dataset)

    # Save the model and tokenizer
    model.save_pretrained(f"./models/{model_name}")
    tokenizer.save_pretrained(f"./models/{model_name}")

    # Save evaluation results
    eval_results['predictions'] = predictions.predictions.tolist()  # Convert to list for serialization
    eval_results['labels'] = val_labels  # Add true labels for confusion matrix generation
    with open(f"./models/{model_name}_eval_results.json", "w") as f:
        json.dump(eval_results, f)

    # Remove the 'predictions' and 'labels' key for logging
    eval_results_to_log = {k: v for k, v in eval_results.items() if k not in ['predictions', 'labels']}
    
    logger.info(f"Evaluation results for {model_name}: {eval_results_to_log}")
    
    # Plot confusion matrix
    predictions = np.argmax(np.array(eval_results['predictions']), axis=1)
    plot_confusion_matrix(eval_results['labels'], predictions, model_name)

    # Log classification report
    report = classification_report(eval_results['labels'], predictions, target_names=[str(i) for i in range(5)])
    logger.info(f"Classification report for {model_name}:\n{report}")

    return model, tokenizer, eval_results


def load_model(model_name):
    base_model_name = model_name.split('_fine_tuned')[0] if '_fine_tuned' in model_name else model_name
    tokenizer = AutoTokenizer.from_pretrained(f"./models/{model_name}")
    model_class = model_names[base_model_name][1]
    model = model_class.from_pretrained(f"./models/{model_name}")

    # Check for the identifier to ensure the correct model is loaded
    if '_fine_tuned' in model_name:
        assert model.config.model_id == model_name, f"Model ID mismatch: expected {model_name}, found {model.config.model_id}"

    # Load evaluation results
    with open(f"./models/{model_name}_eval_results.json", "r") as f:
        eval_results = json.load(f)
    
    # Remove the 'predictions' and 'labels' key for logging
    eval_results_to_log = {k: v for k, v in eval_results.items() if k not in ['predictions', 'labels']}
    
    logger.info(f"Loaded evaluation results for {model_name}: {eval_results_to_log}")
    
    # Plot confusion matrix
    predictions = np.argmax(np.array(eval_results['predictions']), axis=1)
    plot_confusion_matrix(eval_results['labels'], predictions, model_name)
    
    # Log classification report
    report = classification_report(eval_results['labels'], predictions, target_names=[str(i) for i in range(5)])
    logger.info(f"Classification report for {model_name}:\n{report}")
    
    return model, tokenizer, eval_results

def objective(trial):
    model_name = best_model_name  # Use the best model selected from initial training
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1, step=0.01)
    _, _, eval_results = train_model(model_name, train_df, learning_rate, weight_decay=weight_decay)
    return eval_results['eval_f1']

# Initial training and evaluation
models = {}
results = {}
for model_name in model_names.keys():
    model_dir = f"./models/{model_name}"
    if os.path.exists(model_dir):
        logger.info(f"Loading existing model: {model_name}")
        model, tokenizer, eval_results = load_model(model_name)
        if 'eval_f1' not in eval_results:
            logger.info(f"Re-evaluating model: {model_name} to include 'eval_f1' metric.")
            torch.cuda.empty_cache()
            model, tokenizer, eval_results = train_model(model_name, train_df)
    else:
        # Clear CUDA cache before training each model
        torch.cuda.empty_cache()
        model, tokenizer, eval_results = train_model(model_name, train_df)
    models[model_name] = (model, tokenizer)
    results[model_name] = eval_results

if not results:
    logger.info("No models were trained. Exiting.")
    exit()

# Verify that eval_f1 score is in results
for model_name, eval_results in results.items():
    if 'eval_f1' not in eval_results:
        raise KeyError(f"Model {model_name} evaluation results do not contain 'eval_f1' metric.")

# Select the best model based on eval_f1 score
best_model_name = max(results, key=lambda k: results[k]['eval_f1'])
logger.info(f"Best model: {best_model_name} with F1 score: {results[best_model_name]['eval_f1']}")


INFO:Sentiment-Analysis:Loading existing model: DistilBERT
INFO:Sentiment-Analysis:Loaded evaluation results for DistilBERT: {'eval_accuracy': 0.611, 'eval_precision': 0.613907435495592, 'eval_recall': 0.611, 'eval_f1': 0.6119259907555532, 'eval_loss': 0.9109187126159668, 'eval_runtime': 6.8266, 'eval_samples_per_second': 292.97, 'eval_steps_per_second': 36.621, 'epoch': 3.0}


INFO:Sentiment-Analysis:Classification report for DistilBERT:
              precision    recall  f1-score   support

           0       0.76      0.73      0.75       407
           1       0.55      0.59      0.57       395
           2       0.55      0.50      0.53       389
           3       0.51      0.54      0.52       398
           4       0.70      0.68      0.69       411

    accuracy                           0.61      2000
   macro avg       0.61      0.61      0.61      2000
weighted avg       0.61      0.61      0.61      2000

INFO:Sentiment-Analysis:Loading existing model: Electra
INFO:Sentiment-Analysis:Loaded evaluation results for Electra: {'eval_accuracy': 0.673, 'eval_precision': 0.671377890874433, 'eval_recall': 0.673, 'eval_f1': 0.671650215315836, 'eval_loss': 0.7965583205223083, 'eval_runtime': 13.431, 'eval_samples_per_second': 148.91, 'eval_steps_per_second': 18.614, 'epoch': 3.0}


INFO:Sentiment-Analysis:Classification report for Electra:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       407
           1       0.62      0.60      0.61       395
           2       0.64      0.58      0.61       389
           3       0.57      0.59      0.58       398
           4       0.72      0.79      0.75       411

    accuracy                           0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000

INFO:Sentiment-Analysis:Loading existing model: RoBERTa
INFO:Sentiment-Analysis:Loaded evaluation results for RoBERTa: {'eval_accuracy': 0.652, 'eval_precision': 0.652719605818354, 'eval_recall': 0.652, 'eval_f1': 0.6519977059556552, 'eval_loss': 0.8342633247375488, 'eval_runtime': 12.9223, 'eval_samples_per_second': 154.771, 'eval_steps_per_second': 19.346, 'epoch': 3.0}


INFO:Sentiment-Analysis:Classification report for RoBERTa:
              precision    recall  f1-score   support

           0       0.78      0.79      0.78       407
           1       0.60      0.59      0.60       395
           2       0.59      0.55      0.57       389
           3       0.55      0.60      0.58       398
           4       0.73      0.72      0.72       411

    accuracy                           0.65      2000
   macro avg       0.65      0.65      0.65      2000
weighted avg       0.65      0.65      0.65      2000

INFO:Sentiment-Analysis:Best model: Electra with F1 score: 0.671650215315836


In [15]:
# Clear CUDA cache before fine-tuning
torch.cuda.empty_cache()

# Check if the fine-tuned model exists
fine_tuned_model_dir = f"./models/{best_model_name}_fine_tuned"
if os.path.exists(fine_tuned_model_dir):
    logger.info(f"Loading existing fine-tuned model: {best_model_name}")
    best_model, best_tokenizer, best_eval_results = load_model(f"{best_model_name}_fine_tuned")
else:
    # Fine-tune the best model using Optuna
    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=10)

    best_trial = study.best_trial
    logger.info(f"Best trial: {best_trial.params}")

    # Train the best model with the best hyperparameters
    best_learning_rate = best_trial.params['learning_rate']
    best_weight_decay = best_trial.params['weight_decay']
    best_model, best_tokenizer, best_eval_results = train_model(best_model_name, train_df, best_learning_rate, use_early_stopping=True, weight_decay=best_weight_decay)

    # Add an identifier to the model config for the fine-tuned model
    best_model.config.model_id = f"{best_model_name}_fine_tuned"

    # Save the fine-tuned model and evaluation results
    best_model.save_pretrained(fine_tuned_model_dir)
    best_tokenizer.save_pretrained(fine_tuned_model_dir)
    with open(f"./models/{best_model_name}_fine_tuned_eval_results.json", "w") as f:
        json.dump(best_eval_results, f)


INFO:Sentiment-Analysis:Loading existing fine-tuned model: Electra
INFO:Sentiment-Analysis:Loaded evaluation results for Electra_fine_tuned: {'eval_accuracy': 0.6775, 'eval_precision': 0.6755231961374651, 'eval_recall': 0.6775, 'eval_f1': 0.6760792597253367, 'eval_loss': 0.787642240524292, 'eval_runtime': 13.005, 'eval_samples_per_second': 153.787, 'eval_steps_per_second': 19.223, 'epoch': 3.0}


INFO:Sentiment-Analysis:Classification report for Electra_fine_tuned:
              precision    recall  f1-score   support

           0       0.79      0.81      0.80       407
           1       0.64      0.61      0.62       395
           2       0.64      0.60      0.62       389
           3       0.58      0.58      0.58       398
           4       0.73      0.78      0.75       411

    accuracy                           0.68      2000
   macro avg       0.67      0.68      0.67      2000
weighted avg       0.68      0.68      0.68      2000



In [16]:
def predict(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    model.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).item()
    return predictions

# Gradio interface
def predict_sample(sample_index):
    sample = test_df.iloc[int(sample_index.split(":")[0])]
    text = sample['text']
    ground_truth = sample['label']

    predictions = {}
    for model_name, (model, tokenizer) in models.items():
        predictions[model_name] = predict(model, tokenizer, text)
    
    predictions['Fine-Tuned'] = predict(best_model, best_tokenizer, text)
    
    return {
        "text": text,
        "ground_truth": ground_truth,
        "predictions": predictions
    }

# Create choices for dropdown with index and first few words of text
choices = [f"{i}: {test_df.iloc[i]['text'][:30]}..." for i in range(len(test_df))]

sample_dropdown = gr.Dropdown(choices=choices, label="Select a Sample")

iface = gr.Interface(
    fn=predict_sample,
    inputs=sample_dropdown,
    outputs="json",
    description="Sentiment Analysis Prediction"
)

iface.launch()

INFO:httpx:HTTP Request: GET http://127.0.0.1:7861/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "


Running on local URL:  http://127.0.0.1:7861


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"



To create a public link, set `share=True` in `launch()`.
